In [2]:
from transitions import Machine
import re

class Part:
    def __init__(self, ratings):
        self.ratings = ratings

class WorkflowMachine:
    def __init__(self, workflows):
        self.workflows = self.parse_workflows(workflows)
        self.machine = Machine(model=self, states=list(self.workflows.keys()), initial='in')
        self.add_transitions()

    def parse_workflows(self, workflows):
        workflow_dict = {}
        for workflow in workflows:
            name, rules_str = workflow.split('{')
            rules_str = rules_str[:-1]
            rules = rules_str.split(',')
            parsed_rules = []
            for rule in rules:
                condition, action = rule.split(':') if ':' in rule else (None, rule)
                parsed_rules.append((condition, action))
            workflow_dict[name] = parsed_rules
        return workflow_dict

    def add_transitions(self):
        for workflow, rules in self.workflows.items():
            for condition, action in rules:
                if condition:
                    self.machine.add_transition(
                        trigger='process',
                        source=workflow,
                        dest='=' + action,
                        conditions=lambda part, condition=condition: self.check_condition(part, condition)
                    )
                else:
                    self.machine.add_transition(
                        trigger='process',
                        source=workflow,
                        dest=action
                    )

    def check_condition(self, part, condition):
        match = re.match(r'([xmas])\s*([<>])\s*(\d+)', condition)
        if match:
            attr, operator, value = match.groups()
            part_value = part.ratings[attr]
            value = int(value)
            return (part_value > value if operator == '>' else part_value < value)
        return False

    def process_part(self, part):
        while self.state not in ['A', 'R']:
            self.process(part)
        return self.state

workflows = [
    'px{a<2006:qkq,m>2090:A,rfg}',
    'pv{a>1716:R,A}',
    'lnx{m>1548:A,A}',
    'rfg{s<537:gd,x>2440:R,A}',
    'qs{s>3448:A,lnx}',
    'qkq{x<1416:A,crn}',
    'crn{x>2662:A,R}',
    'in{s<1351:px,qqz}',
    'qqz{s>2770:qs,m<1801:hdj,R}',
    'gd{a>3333:R,R}',
    'hdj{m>838:A,pv}',
]

parts = [ 
    '{x=787,m=2655,a=1222,s=2876}', 
    '{x=1679,m=44,a=2067,s=496}',  
    '{x=2036,m=264,a=79,s=2244}',  
    '{x=2461,m=1339,a=466,s=291}', 
    '{x=2127,m=1623,a=2188,s=1013}' 
    ]

def parse_part(part_str):
    ratings = re.findall(r'([xmas])=(\d+)', part_str)
    return {k: int(v) for k, v in ratings}


In [8]:

machine = WorkflowMachine(workflows)
print(machine.state)
print(machine.workflows)
accepted_parts_sum = 0

for part_str in parts:
    part_ratings = parse_part(part_str)
    print(part_ratings)
    part = Part(part_ratings)
    print(part.ratings)
    result = machine.process_part(part)
    print(result)
    if result == 'A':
        accepted_parts_sum += sum(part_ratings.values())

print(accepted_parts_sum)


in
{'px': [('a<2006', 'qkq'), ('m>2090', 'A'), (None, 'rfg')], 'pv': [('a>1716', 'R'), (None, 'A')], 'lnx': [('m>1548', 'A'), (None, 'A')], 'rfg': [('s<537', 'gd'), ('x>2440', 'R'), (None, 'A')], 'qs': [('s>3448', 'A'), (None, 'lnx')], 'qkq': [('x<1416', 'A'), (None, 'crn')], 'crn': [('x>2662', 'A'), (None, 'R')], 'in': [('s<1351', 'px'), (None, 'qqz')], 'qqz': [('s>2770', 'qs'), ('m<1801', 'hdj'), (None, 'R')], 'gd': [('a>3333', 'R'), (None, 'R')], 'hdj': [('m>838', 'A'), (None, 'pv')]}
{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}
{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}


ValueError: State '=qs' is not a registered state.